In [1]:
%matplotlib inline

import scipy.io
import matplotlib.pyplot as plt
import scipy
import numpy as np
import os
import itertools

In [2]:
%matplotlib inline

import mpld3
mpld3.enable_notebook()

In [3]:
import numpy

def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError, "smooth only accepts 1 dimension arrays."

    if x.size < window_len:
        raise ValueError, "Input vector needs to be bigger than window size."


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y[(window_len/2-1):-(window_len/2)]


In [4]:
from scipy.signal import argrelmax, argrelmin, find_peaks_cwt

def findLocalExtremeIndices(sig, widthPercentage = 1.0):
    peaks = find_peaks_cwt(sig, np.arange(1,len(sig)*(widthPercentage/100)))
    maximums = argrelmax(sig)[0]
    maximums = map(lambda peak: min(maximums, key=lambda x:abs(x-peak)), peaks)
    minimums = argrelmin(sig)[0]
    minimums = map(lambda peak: min(minimums, key=lambda x:abs(x-peak)), peaks)
    
    minimums = list(set(minimums))
    maximums = list(set(maximums))
    minimums.sort()
    maximums.sort()
    
    return minimums, maximums

In [5]:
def get_cell_type(cell):
    num = cell['CellNum'][0][0][0][0]
    if num in [19,20,21,22,23,24,36,37,38]:
        return 'SpVi'
    else:
        return 'PrV'

In [6]:
def compute_xvalue_averages_for_one_cell(cell):
    ''' vMat
    A matrix that represesnts the potential responses of the cell.
    80 trials, each 40,000 samples long
    vMat[i][j] := potential in sample i at trial j (mV)

    40,000 samples \ 20KHz sampling rate -> trials are 2 seconds long.

    thus each sample is 2 \ 40000 => 1  \ 20000 => 0.05 ms long
    '''
    vMat = cell['vMat'][0][0].transpose()
    
    ''' stimMat
    A matrix that the galvo feedback (the stimilus to the electical device 
    that mechanically stimulated the whiskers.)
    80 trials, each 8,000 samples long
    stimMat[i][j] := ? amount of stimulus

    8,000 samples \ 4KHz sampling rate -> trials are 2 seconds long.
    thus each sample is 2 \ 8000 = 1 \ 4000 = 0.25ms long
    
     Note that vMat is 5x in the X axis.

    '''
    stimMat = cell['stimMat'][0][0].transpose()
    xValuesVec = cell['xValue'][0][0].tolist()
    
    '''
    xValue is a unique identifier for a stimilus pattern.
    There are 20 (as you can see below) xValues.
    Each trial was done in one of 20 stimulus patterns, and we can check which by using the xValueVec.
    '''
    xValues = np.unique(xValuesVec)
    
    xValueToTrials = dict(map(lambda xValue: (xValue, [i for i in range(0,len(xValuesVec)) if xValuesVec[i] == xValue]), xValues))
    xValueToAverageVoltagesVector = dict(map(lambda xValue: (xValue, np.mean(vMat[xValueToTrials[xValue]], axis = 0)), xValueToTrials))
    xValueToAverageStimulusVector = dict(map(lambda xValue: (xValue, np.mean(stimMat[xValueToTrials[xValue]], axis = 0)), xValueToTrials))

    return (xValueToAverageStimulusVector,xValueToAverageVoltagesVector)

In [7]:
get_cell_type

<function __main__.get_cell_type>

In [8]:
'''
    Overall average computation
'''
filenames = [os.path.join('data', fn) for fn in os.walk('data').next()[2] if fn.endswith('.mat')]
mats = [scipy.io.loadmat(fn) for fn in filenames]
cells = [mat['Cell'] for mat in mats]
xValueAverageVectors = [compute_xvalue_averages_for_one_cell(cell) for cell in cells]

In [9]:
# '''
#     EXAMPLES
#     the following 4 lines are an example for one xvalue of one cell
# '''
# exampleXValue = 1
# (xValueToAverageStimulusVector, xValueToAverageVoltagesVector) =  xValueAverageVectors[0]
# stimsample = xValueToAverageStimulusVector[exampleXValue]
# plt.plot(stimsample)
# plt.show()

In [10]:
# stimsamplesmooth = smooth(stimsample, window_len = 100,window='hanning')
# plt.plot(stimsamplesmooth)
# plt.show()

In [11]:
# vsample = xValueToAverageVoltagesVector[exampleXValue]
# plt.plot(vsample)
# plt.show()

In [12]:
# vsamplesmooth = smooth(vsample, window_len = 500,window='hanning')
# plt.plot(vsamplesmooth)
# plt.show()

In [13]:
spviAverages = [avgs for cell, avgs in itertools.izip(cells, xValueAverageVectors) if get_cell_type(cell) == 'SpVi']
prvAverages = [avgs for cell, avgs in itertools.izip(cells, xValueAverageVectors) if get_cell_type(cell) == 'PrV']

In [29]:
spviXValueToStimulusAverages = map(lambda xvalue: [stimAvgVec[xvalue] for stimAvgVec, voltAvgVec in spviAverages], range(1,21))
spviXValueToVoltageAverages = map(lambda xvalue: [voltAvgVec[xvalue] for stimAvgVec, voltAvgVec in spviAverages], range(1,21))

prvXValueToStimulusAverages = map(lambda xvalue: [stimAvgVec[xvalue] for stimAvgVec, voltAvgVec in prvAverages], range(1,21))
prvXValueToVoltageAverages = map(lambda xvalue: [voltAvgVec[xvalue] for stimAvgVec, voltAvgVec in prvAverages], range(1,21))

In [30]:
spviXValueToStimulusOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in spviXValueToStimulusAverages]
spviXValueToVoltageOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in spviXValueToVoltageAverages]

prvXValueToStimulusOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in prvXValueToStimulusAverages]
prvXValueToVoltageOverallAverageVectors = [smooth(np.mean(v, axis = 0), window_len = 100,window='hanning') for v in prvXValueToVoltageAverages]

In [31]:
for xvalue in range(0,20):
    plt.close(1)
    plt.rc('xtick', labelsize=2.65)
    plt.plot([x * 0.25 for x in range(0, len(spviXValueToStimulusOverallAverageVectors[xvalue]))], spviXValueToStimulusOverallAverageVectors[xvalue])
    plt.xticks(np.arange(0, (len(spviXValueToStimulusOverallAverageVectors[xvalue])+1)*0.25, 50.0))
    plt.savefig(r"C:\Users\scaly\Documents\data\graphs\spvi_stim_%d.pdf" % (xvalue+1))
    plt.close(1)
   
    plt.rc('xtick', labelsize=2.65)
    plt.plot([x * 0.05 for x in range(0, len(spviXValueToVoltageOverallAverageVectors[xvalue]))], spviXValueToVoltageOverallAverageVectors[xvalue])
    plt.xticks(np.arange(0, (len(spviXValueToVoltageOverallAverageVectors[xvalue])+1)*0.05, 50.0))
    plt.savefig(r"C:\Users\scaly\Documents\data\graphs\spvi_volt_%d.pdf" % (xvalue+1))
    plt.close(1)
   
    plt.rc('xtick', labelsize=2.65)
    plt.plot([x * 0.25 for x in range(0, len(prvXValueToStimulusOverallAverageVectors[xvalue]))], prvXValueToStimulusOverallAverageVectors[xvalue])
    plt.xticks(np.arange(0, (len(prvXValueToStimulusOverallAverageVectors[xvalue])+1)*0.25, 50.0))
    plt.savefig(r"C:\Users\scaly\Documents\data\graphs\prv_stim_%d.pdf" % (xvalue+1))
   
    plt.close(1)
    plt.rc('xtick', labelsize=2.65)
    plt.plot([x * 0.05 for x in range(0, len(prvXValueToVoltageOverallAverageVectors[xvalue]))], prvXValueToVoltageOverallAverageVectors[xvalue])
    plt.xticks(np.arange(0, (len(prvXValueToVoltageOverallAverageVectors[xvalue])+1)*0.05, 50.0))
    plt.savefig(r"C:\Users\scaly\Documents\data\graphs\prv_volt_%d.pdf" % (xvalue+1))
    plt.close(1)

In [23]:
# xvalue = 1
# plt.close(1)
# plt.rc('xtick', labelsize=2.7) 
# plt.plot([x * 0.25 for x in range(0, len(spviXValueToStimulusOverallAverageVectors[xvalue]))], spviXValueToStimulusOverallAverageVectors[xvalue])
# plt.xticks(np.arange(0, len(spviXValueToStimulusOverallAverageVectors[xvalue]+1)*0.25, 50.0))
# plt.savefig(r"C:\Users\scaly\Documents\data\graphs\spvi_stim_%d.pdf" % (xvalue + 1), format = "pdf")

In [24]:
xvalue = 1
plt.close(1)
plt.rc('xtick', labelsize=2.8) 
plt.plot([x * 0.05 for x in range(0, len(spviXValueToVoltageOverallAverageVectors[xvalue]))], spviXValueToVoltageOverallAverageVectors[xvalue])
plt.xticks(np.arange(0, len(spviXValueToVoltageOverallAverageVectors[xvalue]+1)*0.05, 50.0))
plt.savefig(r"C:\Users\scaly\Documents\data\graphs\spvi_stim_%d.pdf" % (xvalue + 1), format = "pdf")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [19]:
len(spviXValueToStimulusOverallAverageVectors[1][0])

TypeError: object of type 'numpy.float64' has no len()